In [1]:
# 1. PDF downloader
#-------------------------------------------------------------------------------------------------------------------------------

import os  # For file and directory manipulation, for interacting with the operating system
import random  # To generate random numbers
from selenium import webdriver  # For automating web browsers
from selenium.webdriver.common.by import By  # To locate elements on a webpage
from selenium.webdriver.support.ui import WebDriverWait  # To wait until certain conditions are met on a webpage.
from selenium.webdriver.support import expected_conditions as EC  # To define expected conditions
from selenium.common.exceptions import StaleElementReferenceException  # To handle exceptions related to elements on the webpage that are no longer available.
import pygame # Allows you to handle graphics, sounds and input events.
from webdriver_manager.chrome import ChromeDriverManager # To avoid compatibility issues with the ChromeDrive version of ChromeDrive

import shutil # Used for high-level file operations, such as copying, moving, renaming, and deleting files and directories.


# 2. Generate PDF input with key tables
#-------------------------------------------------------------------------------------------------------------------------------

import fitz  # This library is used for working with PDF documents, including reading, writing, and modifying PDFs (PyMuPDF).
import tkinter as tk  # This library is used for creating graphical user interfaces (GUIs) in Python.


# 3. Data cleaning
#-------------------------------------------------------------------------------------------------------------------------------

# 3.1. A brief documentation on issus in the table information of the PDFs

from PIL import Image  # Used for opening, manipulating, and saving image files.
import matplotlib.pyplot as plt  # Used for creating static, animated, and interactive visualizations.

# 3.2. Extracting tables and data cleanup

import pdfplumber  # For extracting text and metadata from PDF files
import pandas as pd  # For data manipulation and analysis
import unicodedata  # For manipulating Unicode data
import re  # For regular expressions operations
from datetime import datetime  # For working with dates and times
import locale  # For locale-specific formatting of numbers, dates, and currencies

# 3.2.1. Table 1. Extraction and cleaning of data from tables on monthly real GDP growth rates.

import tabula  # Used to extract tables from PDF files into pandas DataFrames
from tkinter import Tk, messagebox, TOP, YES, NO  # Used for creating graphical user interfaces
from sqlalchemy import create_engine  # Used for connecting to and interacting with SQL databases

# 3.2.2. Table 2. Extraction and cleaning of data from tables on quarterly and annual real GDP growth rates.

import roman
from datetime import datetime


# 4. Real-time data of Peru's GDP growth rates
#-------------------------------------------------------------------------------------------------------------------------------

import psycopg2  # For interacting with PostgreSQL databases
from sqlalchemy import create_engine, text  # For creating and executing SQL queries using SQLAlchemy


pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

In [2]:
# Folder path to save downloaded PDF files

raw_pdf = 'raw_pdf' # to save raw data (.pdf).
if not os.path.exists(raw_pdf):
    os.mkdir(raw_pdf) # to create the folder (if it doesn't exist)

In [3]:
# Folder path to save text file with the names of already downloaded files

download_record = 'download_record'
if not os.path.exists(download_record):
    os.mkdir(download_record) # to create the folder (if it doesn't exist)

In [4]:
# Folder path to download the trimmed PDF files (these are PDF inputs for the extraction and cleanup code)

input_pdf = 'input_pdf'
if not os.path.exists(input_pdf):
    os.makedirs(input_pdf) # to create the folder (if it doesn't exist)

In [5]:
# Folder path to save PDF files containing only the pages of interest (where the GDP growth rate tables are located)

input_pdf_record = 'input_pdf_record'
if not os.path.exists(input_pdf_record):
    os.makedirs(input_pdf_record)

In [6]:
# Folder path to save dataframes generated record by year

dataframes_record = 'dataframes_record'
if not os.path.exists(dataframes_record):
    os.makedirs(dataframes_record) # to create the folder (if it doesn't exist)

In [7]:
# Folder path to save sound files

sound_folder = 'sound'
if not os.path.exists(sound_folder):
    os.makedirs(sound_folder) # to create the folder (if it doesn't exist)

In [8]:
# Folder path to save screenshots about issues in the table information

ns_issues_folder = 'ns_issues_folder'
if not os.path.exists(ns_issues_folder):
    os.makedirs(ns_issues_folder) # to create the folder (if it doesn't exist)

In [9]:
def create_sqlalchemy_engine():
    """
    Function to create an SQLAlchemy engine using environment variables.
    
    Returns:
        engine: SQLAlchemy engine object.
    """
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')  # Get the SQL user from environment variables
    password = os.environ.get('CIUP_SQL_PASS')  # Get the SQL password from environment variables
    host = os.environ.get('CIUP_SQL_HOST')  # Get the SQL host from environment variables
    port = 5432  # Set the SQL port to 5432
    database = 'gdp_revisions_datasets'  # Set the database name 'gdp_revisions_datasets' from SQL

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    
    return engine

In [10]:
from new_gdp_datasets_functions import * 

In [ ]:
# Setting the BCRP URL
bcrp_url = "https://www.bcrp.gob.pe/publicaciones/nota-semanal.html"  # Never replace this URL

In [ ]:
# Initialize pygame
pygame.mixer.init()

# List of available sound files
available_sounds = os.listdir(sound_folder)

# Select a random sound
random_sound = random.choice(available_sounds)

# Full path of the random sound
sound_path = os.path.join(sound_folder, random_sound)

# Load the selected sound
pygame.mixer.music.load(sound_path)

# List to keep track of successfully downloaded files
downloaded_files = []

# Load the list of previously downloaded files if it exists
if os.path.exists(os.path.join(download_record, "downloaded_files.txt")):
    with open(os.path.join(download_record, "downloaded_files.txt"), "r") as f:
        downloaded_files = f.read().splitlines()

# Web driver setup

'''
Nota: Download chrome.exe from 'https://googlechromelabs.github.io/chrome-for-testing/#stable'
and call in (1) the folder where you saved this application.
'''
driver = webdriver.Chrome(ChromeDriverManager().install())  # This automatically handles the correct ChromeDriver version

# Number of downloads per batch
downloads_per_batch = 5
# Total number of downloads
total_downloads = 5

try:
    # Open the test page
    driver.get(bcrp_url)
    print("Site opened successfully")

    # Wait for the container area to be present
    wait = WebDriverWait(driver, 60)
    container_area = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="rightside"]')))

    # Get all the links within the container area
    pdf_links = container_area.find_elements(By.XPATH, './/a')

    # Reverse the order of links
    pdf_links = list(reversed(pdf_links))

    # Initialize download counter
    download_counter = 0

    # Iterate over reversed links and download PDFs in batches
    for pdf_link in pdf_links:
        download_counter += 1

        # Get the file name from the URL
        new_url = pdf_link.get_attribute("href")
        file_name = new_url.split("/")[-1]

        # Check if the file has already been downloaded
        if file_name in downloaded_files:
            print(f"{download_counter}. The file {file_name} has already been downloaded previously. Skipping...")
            continue

        # Try to download the file
        try:
            download_pdf(driver, pdf_link, wait, download_counter, raw_pdf, download_record)

            # Update the list of downloaded files
            downloaded_files.append(file_name)

        except Exception as e:
            print(f"Error downloading the file {file_name}: {str(e)}")

        # If the download count reaches a multiple of batch size, notify
        if download_counter % downloads_per_batch == 0:
            print(f"Batch {download_counter // downloads_per_batch} of {downloads_per_batch} completed")

        # If the download count reaches a multiple of 25, ask the user if they want to continue
        if download_counter % 5 == 0: # after the fifth PDF downloaded, you'll listen a beautiful song
            play_sound()
            user_input = input("Do you want to continue downloading? (Enter 'y' to continue, any other key to stop): ")
            pygame.mixer.music.stop()
            if user_input.lower() != 'y':
                break

        # Random wait before the next iteration
        random_wait(5, 10)

        # If total downloads reached, break out of loop
        if download_counter == total_downloads:
            print(f"All downloads completed ({total_downloads} in total)")
            break

except StaleElementReferenceException:
    print("StaleElementReferenceException occurred. Retrying...")

finally:
    # Close the browser when finished
    driver.quit()


In [ ]:
# Get the list of files in the directory
files = os.listdir(raw_pdf)

# Call the function to organize files
organize_files_by_year(raw_pdf)

In [ ]:
# The code will reduce the number of pages of each PDF saved in the folder 'raw_pdf'.
if __name__ == "__main__":
    keywords = ["ECONOMIC SECTORS"] # Keywords to identify the 2 key tables or pages within the 100+ page PDF.
    root = tk.Tk()
    root.withdraw()  # Hide the main Tkinter window

    input_pdf_files = read_input_pdf_files()
    processing_counter = 1

    for folder in os.listdir(raw_pdf):
        folder_path = os.path.join(raw_pdf, folder)
        if os.path.isdir(folder_path):
            print("Processing folder:", folder)
            num_pdfs_trimmed = 0
            for filename in os.listdir(folder_path):
                if filename.endswith(".pdf"):
                    pdf_file = os.path.join(folder_path, filename)
                    if filename in input_pdf_files:
                        print(f"{processing_counter}. The PDF '{filename}' has already been trimmed and saved in '{input_pdf}'...")
                        processing_counter += 1
                        continue
                    print(f"{processing_counter}. Processing:", pdf_file)
                    
                    pages_with_keywords = search_keywords(pdf_file, keywords)
                    num_pages_new_pdf = trim_pdf(pdf_file, pages_with_keywords)
                    if num_pages_new_pdf > 0:
                        num_pdfs_trimmed += 1
                        input_pdf_files.add(filename)
                        processing_counter += 1
            
            write_input_pdf_files(input_pdf_files)

            message = f"{num_pdfs_trimmed} PDFs have been trimmed in folder {folder}. Do you want to continue?"
            popup = PopupWindow(root, message)
            root.wait_window(popup)
            if not popup.result:
                break
                
    print("Process completed for all PDFs in directory:", input_pdf)


In [ ]:
# Get the list of files in the directory
files = os.listdir(input_pdf)

# Call the function to organize files
organize_files_by_year(input_pdf)

In [12]:
# Set the locale to Spanish
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Dictionary to store generated DataFrames
new_dataframes_dict_1 = {}

# Path for the processed folders log file
record_path = 'dataframes_record/new_processed_folders_1.txt'

# Function to correct month names
def correct_month_name(month):
    months_mapping = {
        'setiembre': 'septiembre',
        # Add more mappings as needed for other month names
    }
    return months_mapping.get(month, month)

# Function to register processed folder
def register_processed_folder(folder, num_processed_files):
    with open(record_path, 'a') as file:
        file.write(f"{folder}:{num_processed_files}\n")

# Function to check if folder has been processed
def folder_processed(folder):
    if not os.path.exists(record_path):
        return False
    with open(record_path, 'r') as file:
        for line in file:
            if line.startswith(folder):
                return True
    return False

# Function to fetch date from database
def get_date(df, engine):
    id_ns = df['id_ns'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE id_ns = '{id_ns}' AND year = '{year}';"
    date_result = pd.read_sql(query, engine)
    return date_result.iloc[0, 0] if not date_result.empty else None

# Function to process PDF file
def process_pdf(pdf_path):
    new_tables_dict_1 = {}  # Local dictionary for each PDF
    table_counter = 1
    keyword_count = 0 

    filename = os.path.basename(pdf_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        id_ns, year = id_ns_year_matches[0]
    else:
        print("No matches found for id_ns and year in filename:", filename)
        return None, None, None, None, None  # Return None for new_tables_dict_1 as well

    new_filename = os.path.splitext(os.path.basename(pdf_path))[0].replace('-', '_')

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, 1):
            text = page.extract_text()
            if all(keyword in text for keyword in keywords):
                keyword_count += 1
                if keyword_count == 1:  # Process only the first occurrence
                    tables = tabula.read_pdf(pdf_path, pages=i, multiple_tables=False, stream=True) # Change stream to another option if desired
                    for j, table_df in enumerate(tables, start=1):
                        dataframe_name = f"{new_filename}_{keyword_count}"
                        new_tables_dict_1[dataframe_name] = table_df
                        table_counter += 1

                    break  # Exit loop after finding the first occurrence

    return id_ns, year, new_tables_dict_1, keyword_count

# Function to process folder
def process_folder(folder, engine):
    print(f"Processing folder {os.path.basename(folder)}")
    pdf_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.pdf')]

    num_pdfs_processed = 0
    num_dataframes_generated = 0

    table_counter = 1  # Initialize table counter here
    new_tables_dict_1 = {}  # Declare new_tables_dict_1 outside main loop
    
    for pdf_file in pdf_files:
        id_ns, year, tables_dict_temp, keyword_count = process_pdf(pdf_file)

        if tables_dict_temp:
            for dataframe_name, df in tables_dict_temp.items():
                file_name = os.path.splitext(os.path.basename(pdf_file))[0].replace('-', '_')
                dataframe_name = f"{file_name}_{keyword_count}"
                
                # Store raw DataFrame in new_tables_dict_1
                new_tables_dict_1[dataframe_name] = df.copy()
                
                # Apply cleaning functions to a copy of the DataFrame
                df_clean = df.copy()

                if any(col.isdigit() and len(col) == 4 for col in df_clean.columns):
                    # If there is at least one column representing a year
                    df_clean = swap_nan_se(df_clean)
                    df_clean = split_column_by_pattern(df_clean)
                    df_clean = drop_rare_caracter_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = relocate_last_columns(df_clean)
                    df_clean = replace_first_dot(df_clean)
                    df_clean = swap_first_second_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = remove_digit_slash(df_clean)
                    df_clean = replace_var_perc_first_column(df_clean)
                    df_clean = replace_var_perc_last_columns(df_clean)
                    df_clean = replace_number_moving_average(df_clean)
                    df_clean = separate_text_digits(df_clean)
                    df_clean = exchange_values(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = find_year_column(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = get_months_sublist_list(df_clean, year_columns)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_services(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)
                else:
                    # If there are no columns representing years
                    df_clean = check_first_row(df_clean)
                    df_clean = check_first_row_1(df_clean)
                    df_clean = replace_first_row_with_columns(df_clean)
                    df_clean = swap_nan_se(df_clean)
                    df_clean = split_column_by_pattern(df_clean)
                    df_clean = drop_rare_caracter_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = relocate_last_columns(df_clean)
                    df_clean = swap_first_second_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = remove_digit_slash(df_clean)
                    df_clean = replace_var_perc_first_column(df_clean)
                    df_clean = replace_var_perc_last_columns(df_clean)
                    df_clean = replace_number_moving_average(df_clean)
                    df_clean = expand_column(df_clean)
                    df_clean = split_values_1(df_clean)
                    df_clean = split_values_2(df_clean)
                    df_clean = split_values_3(df_clean)
                    df_clean = separate_text_digits(df_clean)
                    df_clean = exchange_values(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = find_year_column(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = get_months_sublist_list(df_clean, year_columns)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_nan_with_previous_column_1(df_clean)
                    df_clean = replace_nan_with_previous_column_2(df_clean)
                    df_clean = replace_nan_with_previous_column_3(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_services(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)
                
                # Add 'year' column to cleaned DataFrame
                df_clean.insert(0, 'year', year)
                
                # Add 'id_ns' column to cleaned DataFrame
                df_clean.insert(1, 'id_ns', id_ns)
                
                # Get corresponding date from database
                date = get_date(df_clean, engine)
                if date:
                    # Add 'date' column to cleaned DataFrame
                    df_clean.insert(2, 'date', date)
                else:
                    print("Date not found in database for id_ns:", id_ns, "and year:", year)
                
                # Store cleaned DataFrame in new_dataframes_dict_1
                new_dataframes_dict_1[dataframe_name] = df_clean

                print(f'  {table_counter}. DataFrame generated for file {pdf_file}: {dataframe_name}')
                num_dataframes_generated += 1
                table_counter += 1  # Increment table counter here
        
        num_pdfs_processed += 1  # Increment number of processed PDFs for each PDF in folder

    return num_pdfs_processed, num_dataframes_generated, new_tables_dict_1

# Function to process folders
def process_folders():
    pdf_folder = 'input_pdf'
    folders = [os.path.join(pdf_folder, d) for d in os.listdir(pdf_folder) if os.path.isdir(os.path.join(pdf_folder, d))]
    
    new_tables_dict_1 = {}  # Initialize new_tables_dict_1 here
    
    for folder in folders:
        if folder_processed(folder):
            print(f"Folder {folder} has already been processed.")
            continue
        
        num_pdfs_processed, num_dataframes_generated, tables_dict_temp = process_folder(folder, engine)
        
        # Update new_tables_dict_1 with values returned from process_folder()
        new_tables_dict_1.update(tables_dict_temp)
        
        register_processed_folder(folder, num_pdfs_processed)

        # Ask user if they want to continue with next folder
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Ensure the messagebox is in front
        message = f"Process {folder} complete. Processed {num_pdfs_processed} PDF(s) and generated {num_dataframes_generated} DataFrame(s). Continue with next folder?"
        if not messagebox.askyesno("Continue?", message):
            break
            
    print("Processing completed for all folders.")  # Add a message to indicate completion
    
    return new_tables_dict_1

if __name__ == '__main__':
    engine = create_sqlalchemy_engine()
    new_tables_dict_1 = process_folders()

Processing folder 2013
  1. DataFrame generated for file input_pdf\2013\ns-01-2013.pdf: ns_01_2013_1
  2. DataFrame generated for file input_pdf\2013\ns-02-2013.pdf: ns_02_2013_1
  3. DataFrame generated for file input_pdf\2013\ns-03-2013.pdf: ns_03_2013_1
  4. DataFrame generated for file input_pdf\2013\ns-04-2013.pdf: ns_04_2013_1
  5. DataFrame generated for file input_pdf\2013\ns-05-2013.pdf: ns_05_2013_1
  6. DataFrame generated for file input_pdf\2013\ns-06-2013.pdf: ns_06_2013_1
  7. DataFrame generated for file input_pdf\2013\ns-07-2013.pdf: ns_07_2013_1
  8. DataFrame generated for file input_pdf\2013\ns-08-2013.pdf: ns_08_2013_1
  9. DataFrame generated for file input_pdf\2013\ns-09-2013.pdf: ns_09_2013_1
  10. DataFrame generated for file input_pdf\2013\ns-10-2013.pdf: ns_10_2013_1
  11. DataFrame generated for file input_pdf\2013\ns-11-2013.pdf: ns_11_2013_1
  12. DataFrame generated for file input_pdf\2013\ns-12-2013.pdf: ns_12_2013_1
  13. DataFrame generated for file inp

  6. DataFrame generated for file input_pdf\2015\ns-06-2015.pdf: ns_06_2015_1
  7. DataFrame generated for file input_pdf\2015\ns-07-2015.pdf: ns_07_2015_1
  8. DataFrame generated for file input_pdf\2015\ns-08-2015.pdf: ns_08_2015_1
  9. DataFrame generated for file input_pdf\2015\ns-09-2015.pdf: ns_09_2015_1
  10. DataFrame generated for file input_pdf\2015\ns-10-2015.pdf: ns_10_2015_1
  11. DataFrame generated for file input_pdf\2015\ns-11-2015.pdf: ns_11_2015_1
  12. DataFrame generated for file input_pdf\2015\ns-12-2015.pdf: ns_12_2015_1
  13. DataFrame generated for file input_pdf\2015\ns-13-2015.pdf: ns_13_2015_1
  14. DataFrame generated for file input_pdf\2015\ns-14-2015.pdf: ns_14_2015_1
  15. DataFrame generated for file input_pdf\2015\ns-15-2015.pdf: ns_15_2015_1
  16. DataFrame generated for file input_pdf\2015\ns-16-2015.pdf: ns_16_2015_1
  17. DataFrame generated for file input_pdf\2015\ns-17-2015.pdf: ns_17_2015_1
  18. DataFrame generated for file input_pdf\2015\ns-18-

  14. DataFrame generated for file input_pdf\2017\ns-14-2017.pdf: ns_14_2017_1
  15. DataFrame generated for file input_pdf\2017\ns-15-2017.pdf: ns_15_2017_1
  16. DataFrame generated for file input_pdf\2017\ns-16-2017.pdf: ns_16_2017_1
  17. DataFrame generated for file input_pdf\2017\ns-17-2017.pdf: ns_17_2017_1
  18. DataFrame generated for file input_pdf\2017\ns-18-2017.pdf: ns_18_2017_1
  19. DataFrame generated for file input_pdf\2017\ns-19-2017.pdf: ns_19_2017_1
  20. DataFrame generated for file input_pdf\2017\ns-20-2017.pdf: ns_20_2017_1
  21. DataFrame generated for file input_pdf\2017\ns-21-2017.pdf: ns_21_2017_1
  22. DataFrame generated for file input_pdf\2017\ns-22-2017.pdf: ns_22_2017_1
  23. DataFrame generated for file input_pdf\2017\ns-23-2017.pdf: ns_23_2017_1
  24. DataFrame generated for file input_pdf\2017\ns-24-2017.pdf: ns_24_2017_1
  25. DataFrame generated for file input_pdf\2017\ns-25-2017.pdf: ns_25_2017_1
  26. DataFrame generated for file input_pdf\2017\ns

  20. DataFrame generated for file input_pdf\2019\ns-20-2019.pdf: ns_20_2019_1
  21. DataFrame generated for file input_pdf\2019\ns-21-2019.pdf: ns_21_2019_1
  22. DataFrame generated for file input_pdf\2019\ns-22-2019.pdf: ns_22_2019_1
  23. DataFrame generated for file input_pdf\2019\ns-23-2019.pdf: ns_23_2019_1
  24. DataFrame generated for file input_pdf\2019\ns-24-2019.pdf: ns_24_2019_1
  25. DataFrame generated for file input_pdf\2019\ns-25-2019.pdf: ns_25_2019_1
  26. DataFrame generated for file input_pdf\2019\ns-26-2019.pdf: ns_26_2019_1
  27. DataFrame generated for file input_pdf\2019\ns-27-2019.pdf: ns_27_2019_1
  28. DataFrame generated for file input_pdf\2019\ns-28-2019.pdf: ns_28_2019_1
  29. DataFrame generated for file input_pdf\2019\ns-29-2019.pdf: ns_29_2019_1
  30. DataFrame generated for file input_pdf\2019\ns-30-2019.pdf: ns_30_2019_1
  31. DataFrame generated for file input_pdf\2019\ns-31-2019.pdf: ns_31_2019_1
  32. DataFrame generated for file input_pdf\2019\ns

  38. DataFrame generated for file input_pdf\2021\ns-38-2021.pdf: ns_38_2021_1
  39. DataFrame generated for file input_pdf\2021\ns-39-2021.pdf: ns_39_2021_1
  40. DataFrame generated for file input_pdf\2021\ns-40-2021.pdf: ns_40_2021_1
  41. DataFrame generated for file input_pdf\2021\ns-41-2021.pdf: ns_41_2021_1
  42. DataFrame generated for file input_pdf\2021\ns-42-2021.pdf: ns_42_2021_1
  43. DataFrame generated for file input_pdf\2021\ns-43-2021.pdf: ns_43_2021_1
  44. DataFrame generated for file input_pdf\2021\ns-44-2021.pdf: ns_44_2021_1
  45. DataFrame generated for file input_pdf\2021\ns-45-2021.pdf: ns_45_2021_1
  46. DataFrame generated for file input_pdf\2021\ns-46-2021.pdf: ns_46_2021_1
Processing folder 2022
  1. DataFrame generated for file input_pdf\2022\ns-01-2022.pdf: ns_01_2022_1
  2. DataFrame generated for file input_pdf\2022\ns-02-2022.pdf: ns_02_2022_1
  3. DataFrame generated for file input_pdf\2022\ns-03-2022.pdf: ns_03_2022_1
  4. DataFrame generated for fil

  9. DataFrame generated for file input_pdf\2024\ns-09-2024.pdf: ns_09_2024_1
  10. DataFrame generated for file input_pdf\2024\ns-10-2024.pdf: ns_10_2024_1
  11. DataFrame generated for file input_pdf\2024\ns-11-2024.pdf: ns_11_2024_1
  12. DataFrame generated for file input_pdf\2024\ns-12-2024.pdf: ns_12_2024_1
  13. DataFrame generated for file input_pdf\2024\ns-13-2024.pdf: ns_13_2024_1
  14. DataFrame generated for file input_pdf\2024\ns-14-2024.pdf: ns_14_2024_1
  15. DataFrame generated for file input_pdf\2024\ns-15-2024.pdf: ns_15_2024_1
  16. DataFrame generated for file input_pdf\2024\ns-16-2024.pdf: ns_16_2024_1
  17. DataFrame generated for file input_pdf\2024\ns-17-2024.pdf: ns_17_2024_1
  18. DataFrame generated for file input_pdf\2024\ns-18-2024.pdf: ns_18_2024_1
  19. DataFrame generated for file input_pdf\2024\ns-19-2024.pdf: ns_19_2024_1
  20. DataFrame generated for file input_pdf\2024\ns-20-2024.pdf: ns_20_2024_1
  21. DataFrame generated for file input_pdf\2024\ns-

In [13]:
new_tables_dict_1.keys()

dict_keys(['ns_01_2013_1', 'ns_02_2013_1', 'ns_03_2013_1', 'ns_04_2013_1', 'ns_05_2013_1', 'ns_06_2013_1', 'ns_07_2013_1', 'ns_08_2013_1', 'ns_09_2013_1', 'ns_10_2013_1', 'ns_11_2013_1', 'ns_12_2013_1', 'ns_13_2013_1', 'ns_14_2013_1', 'ns_15_2013_1', 'ns_16_2013_1', 'ns_17_2013_1', 'ns_18_2013_1', 'ns_19_2013_1', 'ns_20_2013_1', 'ns_21_2013_1', 'ns_22_2013_1', 'ns_23_2013_1', 'ns_24_2013_1', 'ns_25_2013_1', 'ns_26_2013_1', 'ns_27_2013_1', 'ns_28_2013_1', 'ns_29_2013_1', 'ns_30_2013_1', 'ns_31_2013_1', 'ns_32_2013_1', 'ns_33_2013_1', 'ns_34_2013_1', 'ns_35_2013_1', 'ns_36_2013_1', 'ns_37_2013_1', 'ns_38_2013_1', 'ns_39_2013_1', 'ns_40_2013_1', 'ns_41_2013_1', 'ns_42_2013_1', 'ns_43_2013_1', 'ns_44_2013_1', 'ns_45_2013_1', 'ns_46_2013_1', 'ns_47_2013_1', 'ns_48_2013_1', 'ns_49_2013_1', 'ns_50_2013_1', 'ns_01_2014_1', 'ns_02_2014_1', 'ns_03_2014_1', 'ns_04_2014_1', 'ns_05_2014_1', 'ns_06_2014_1', 'ns_07_2014_1', 'ns_08_2014_1', 'ns_09_2014_1', 'ns_10_2014_1', 'ns_11_2014_1', 'ns_12_2014_1

In [14]:
new_dataframes_dict_1.keys()

dict_keys(['ns_01_2013_1', 'ns_02_2013_1', 'ns_03_2013_1', 'ns_04_2013_1', 'ns_05_2013_1', 'ns_06_2013_1', 'ns_07_2013_1', 'ns_08_2013_1', 'ns_09_2013_1', 'ns_10_2013_1', 'ns_11_2013_1', 'ns_12_2013_1', 'ns_13_2013_1', 'ns_14_2013_1', 'ns_15_2013_1', 'ns_16_2013_1', 'ns_17_2013_1', 'ns_18_2013_1', 'ns_19_2013_1', 'ns_20_2013_1', 'ns_21_2013_1', 'ns_22_2013_1', 'ns_23_2013_1', 'ns_24_2013_1', 'ns_25_2013_1', 'ns_26_2013_1', 'ns_27_2013_1', 'ns_28_2013_1', 'ns_29_2013_1', 'ns_30_2013_1', 'ns_31_2013_1', 'ns_32_2013_1', 'ns_33_2013_1', 'ns_34_2013_1', 'ns_35_2013_1', 'ns_36_2013_1', 'ns_37_2013_1', 'ns_38_2013_1', 'ns_39_2013_1', 'ns_40_2013_1', 'ns_41_2013_1', 'ns_42_2013_1', 'ns_43_2013_1', 'ns_44_2013_1', 'ns_45_2013_1', 'ns_46_2013_1', 'ns_47_2013_1', 'ns_48_2013_1', 'ns_49_2013_1', 'ns_50_2013_1', 'ns_01_2014_1', 'ns_02_2014_1', 'ns_03_2014_1', 'ns_04_2014_1', 'ns_05_2014_1', 'ns_06_2014_1', 'ns_07_2014_1', 'ns_08_2014_1', 'ns_09_2014_1', 'ns_10_2014_1', 'ns_11_2014_1', 'ns_12_2014_1

In [15]:
new_tables_dict_1['ns_43_2024_1'].head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,2023,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,2024,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,SECTORES ECONÓMICOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ECONOMIC SECTORS
1,NaN,Sep.,Oct.,Nov.,Dic.,Año,Ene.,Feb.,Mar.,Abr.,May.,Jun.,Jul.,Ago.,Sep.,Ene.-Sep.,NaN
2,Agropecuario 2/,"-7,6","-5,2","3,2","0,6","-2,4","-2,4","-0,2","1,2","24,0","4,8","-0,8","-3,4","-1,8","1,1","2,8 Agriculture and Livestock 2/",NaN
3,Agrícola,"-12,5","-8,8","5,5","0,7","-3,8","-3,7","-0,1","2,2","36,2","6,1","-1,8","-6,2","-4,4","0,4","3,8",Agriculture
4,Pecuario,"0,0","0,4","-0,1","0,5","-0,1","-0,5","-0,3","-0,3","1,1","1,6","1,4","1,6","1,9","2,1","1,0",Livestock


In [16]:
df_1 = new_dataframes_dict_1['ns_43_2024_1']
df_1

,year,id_ns,date,sectores_economicos,economic_sectors,2023_sep,2023_oct,2023_nov,2023_dic,2023_year,2024_ene,2024_feb,2024_mar,2024_abr,2024_may,2024_jun,2024_jul,2024_ago,2024_sep,2024_ene_sep
1,2024,43,2024-11-28,agropecuario,agriculture and livestock,-7.6,-5.2,3.2,0.6,-2.4,-2.4,-0.2,1.2,24.0,4.8,-0.8,-3.4,-1.8,1.1,2.8
2,2024,43,2024-11-28,agricola,agriculture,-12.5,-8.8,5.5,0.7,-3.8,-3.7,-0.1,2.2,36.2,6.1,-1.8,-6.2,-4.4,0.4,3.8
3,2024,43,2024-11-28,pecuario,livestock,0.0,0.4,-0.1,0.5,-0.1,-0.5,-0.3,-0.3,1.1,1.6,1.4,1.6,1.9,2.1,1.0
4,2024,43,2024-11-28,pesca,fishing,11.2,50.4,55.6,-53.4,-21.2,-25.2,-31.5,-35.4,144.1,363.5,56.8,14.9,-27.8,-14.6,29.6
5,2024,43,2024-11-28,mineria e hidrocarburos,mining and fuel,8.8,3.4,7.5,3.7,8.1,4.9,17.3,3.2,-4.0,2.1,-7.2,3.3,8.8,1.1,2.9
6,2024,43,2024-11-28,mineria metalica,metals,7.3,3.5,10.1,4.2,9.3,5.7,18.7,4.7,-4.1,2.7,-7.8,0.9,8.8,1.7,3.1
7,2024,43,2024-11-28,hidrocarburos,fuel,19.6,2.8,-8.6,-0.1,0.7,0.6,8.8,-5.3,-3.6,-1.7,-3.5,21.6,9.0,-3.3,1.9
8,2024,43,2024-11-28,manufactura,manufacturing,-8.5,-3.8,-0.5,-11.6,-6.5,-4.4,-4.2,-10.2,12.1,18.1,0.0,10.9,2.9,2.7,2.9
9,2024,43,2024-11-28,procesadores recursos primarios,based on raw materials,8.9,9.6,10.9,-28.1,-2.3,-16.0,-22.7,-13.6,30.7,68.5,12.0,12.6,-1.0,-2.6,6.4
10,2024,43,2024-11-28,manufactura no primaria,nonprimary,-12.9,-7.6,-4.4,-3.7,-8.0,0.5,3.1,-9.1,5.5,0.8,-4.1,10.3,4.2,4.3,1.6


In [17]:
df_1[(df_1['sectores_economicos'] == 'agropecuario') | (df_1['economic_sectors'] == 'agriculture and livestock')]

,year,id_ns,date,sectores_economicos,economic_sectors,2023_sep,2023_oct,2023_nov,2023_dic,2023_year,2024_ene,2024_feb,2024_mar,2024_abr,2024_may,2024_jun,2024_jul,2024_ago,2024_sep,2024_ene_sep
1,2024,43,2024-11-28,agropecuario,agriculture and livestock,-7.6,-5.2,3.2,0.6,-2.4,-2.4,-0.2,1.2,24.0,4.8,-0.8,-3.4,-1.8,1.1,2.8


In [18]:
# Keywords to search in the page text
keywords = ["ECONOMIC SECTORS"]

In [19]:
# Set the locale to Spanish
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Dictionary to store generated DataFrames
new_dataframes_dict_2 = {}

# Path for the processed folders log file
record_path = 'dataframes_record/new_processed_folders_2.txt'

# Function to correct month names
def correct_month_name(month):
    months_mapping = {
        'setiembre': 'septiembre',
        # Add more mappings as needed for other month names
    }
    return months_mapping.get(month, month)

# Function to register processed folder
def register_processed_folder(folder, num_processed_files):
    with open(record_path, 'a') as file:
        file.write(f"{folder}:{num_processed_files}\n")
        
# Function to check if folder has been processed
def folder_processed(folder):
    if not os.path.exists(record_path):
        return False
    with open(record_path, 'r') as file:
        for line in file:
            if line.startswith(folder):
                return True
    return False

# Function to fetch date from database
def get_date(df, engine):
    id_ns = df['id_ns'].iloc[0]
    year = df['year'].iloc[0]
    query = f"SELECT date FROM dates_growth_rates WHERE id_ns = '{id_ns}' AND year = '{year}';"
    date_result = pd.read_sql(query, engine)
    return date_result.iloc[0, 0] if not date_result.empty else None

# Function to process PDF file
def process_pdf(pdf_path):
    new_tables_dict_2 = {}  # Local dictionary for each PDF
    table_counter = 1
    keyword_count = 0 

    filename = os.path.basename(pdf_path)
    id_ns_year_matches = re.findall(r'ns-(\d+)-(\d{4})', filename)
    if id_ns_year_matches:
        id_ns, year = id_ns_year_matches[0]
    else:
        print("No matches found for id_ns and year in filename:", filename)
        return None, None, None, None

    new_filename = os.path.splitext(os.path.basename(pdf_path))[0].replace('-', '_')

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, 1):
            text = page.extract_text()
            if all(keyword in text for keyword in keywords):
                keyword_count += 1
                if keyword_count == 2:
                    tables = tabula.read_pdf(pdf_path, pages=i, multiple_tables=False)
                    for j, table_df in enumerate(tables, start=1):
                        dataframe_name = f"{new_filename}_{keyword_count}"
                        new_tables_dict_2[dataframe_name] = table_df
                        table_counter += 1

    return id_ns, year, new_tables_dict_2, keyword_count


def process_folder(folder, engine):
    print(f"Processing folder {os.path.basename(folder)}")
    pdf_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.pdf')]

    num_pdfs_processed = 0
    num_dataframes_generated = 0

    table_counter = 1  # Initialize table counter here
    new_tables_dict_2 = {}  # Declare tables_dict outside main loop
    
    for pdf_file in pdf_files:
        id_ns, year, tables_dict_temp, keyword_count = process_pdf(pdf_file)

        if tables_dict_temp:
            for dataframe_name, df in tables_dict_temp.items():
                file_name = os.path.splitext(os.path.basename(pdf_file))[0].replace('-', '_')
                dataframe_name = f"{file_name}_{keyword_count}"

                # Store raw DataFrame in tables_dict
                new_tables_dict_2[dataframe_name] = df.copy()

                # Apply 20 lines of cleaning functions to a copy of the DataFrame
                df_clean = df.copy()
                if df_clean.iloc[0, 0] is np.nan:
                    # Apply 20 lines of cleaning
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = separate_years(df_clean)
                    df_clean = relocate_roman_numerals(df_clean)
                    df_clean = extract_mixed_values(df_clean)
                    df_clean = replace_first_row_nan(df_clean)
                    df_clean = first_row_columns(df_clean)
                    df_clean = swap_first_second_row(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = drop_nan_row(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = split_values(df_clean)
                    df_clean = separate_text_digits(df_clean)
                    df_clean = roman_arabic(df_clean)
                    df_clean = fix_duplicates(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = get_quarters_sublist_list(df_clean, year_columns)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_services(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)
                else:
                    # Apply 15 lines of cleaning
                    df_clean = exchange_roman_nan(df_clean)
                    df_clean = exchange_columns(df_clean)
                    df_clean = drop_nan_columns(df_clean)
                    df_clean = remove_digit_slash(df_clean)
                    df_clean = last_column_es(df_clean)
                    df_clean = swap_first_second_row(df_clean)
                    df_clean = drop_nan_rows(df_clean)
                    df_clean = reset_index(df_clean)
                    year_columns = extract_years(df_clean)
                    df_clean = separate_text_digits(df_clean)
                    df_clean = roman_arabic(df_clean)
                    df_clean = fix_duplicates(df_clean)
                    df_clean = relocate_last_column(df_clean)
                    df_clean = clean_first_row(df_clean)
                    df_clean = get_quarters_sublist_list(df_clean, year_columns)
                    df_clean = first_row_columns(df_clean)
                    df_clean = clean_columns_values(df_clean)
                    df_clean = reset_index(df_clean)
                    df_clean = convert_float(df_clean)
                    df_clean = replace_set_sep(df_clean)
                    df_clean = spaces_se_es(df_clean)
                    df_clean = replace_services(df_clean)
                    df_clean = replace_mineria(df_clean)
                    df_clean = replace_mining(df_clean)
                    df_clean = rounding_values(df_clean, decimals=1)

                # Add 'year' column to cleaned DataFrame
                df_clean.insert(0, 'year', year)
                
                # Add 'id_ns' column to cleaned DataFrame
                df_clean.insert(1, 'id_ns', id_ns)
                
                # Get corresponding date from database
                date = get_date(df_clean, engine)
                if date:
                    # Add 'date' column to cleaned DataFrame
                    df_clean.insert(2, 'date', date)
                else:
                    print("Date not found in database for id_ns:", id_ns, "and year:", year)

                # Store cleaned DataFrame in new_dataframes_dict
                new_dataframes_dict_2[dataframe_name] = df_clean

                print(f'  {table_counter}. DataFrame generated for file {pdf_file}: {dataframe_name}')
                num_dataframes_generated += 1
                table_counter += 1  # Increment table counter here
                    
        num_pdfs_processed += 1  # Increment number of PDFs processed for each PDF in folder

    return num_pdfs_processed, num_dataframes_generated, new_tables_dict_2
        
def process_folders():
    pdf_folder = 'input_pdf'
    folders = [os.path.join(pdf_folder, d) for d in os.listdir(pdf_folder) if os.path.isdir(os.path.join(pdf_folder, d))]

    new_tables_dict_2 = {}  # Initialize tables_dict here
    
    for folder in folders:
        if folder_processed(folder):
            print(f"Folder {folder} has already been processed.")
            continue
        
        num_pdfs_processed, num_dataframes_generated, tables_dict_temp = process_folder(folder, engine)
        
        # Update tables_dict with values returned from process_folder()
        new_tables_dict_2.update(tables_dict_temp)
        
        register_processed_folder(folder, num_pdfs_processed)
        
        # Ask user if they want to continue with next folder
        root = Tk()
        root.withdraw()
        root.attributes('-topmost', True)  # Ensure the messagebox is in front
        message = f"Process {folder} complete. Processed {num_pdfs_processed} PDF(s) and generated {num_dataframes_generated} dataframes. Continue with next folder?"
        if not messagebox.askyesno("Continue?", message):
            break
            
    print("Processing completed for all folders.")  # Add a message to indicate completion

    return new_tables_dict_2  # Return tables_dict at the end of the function
    
if __name__ == "__main__":
    engine = create_sqlalchemy_engine() # Creates the SQL connection to merge the date, year and id from a SQL database to dataframes
    new_tables_dict_2 = process_folders()  # Capture the returned value from process_folders

Processing folder 2013
  1. DataFrame generated for file input_pdf\2013\ns-01-2013.pdf: ns_01_2013_2
  2. DataFrame generated for file input_pdf\2013\ns-02-2013.pdf: ns_02_2013_2
  3. DataFrame generated for file input_pdf\2013\ns-03-2013.pdf: ns_03_2013_2
  4. DataFrame generated for file input_pdf\2013\ns-04-2013.pdf: ns_04_2013_2
  5. DataFrame generated for file input_pdf\2013\ns-05-2013.pdf: ns_05_2013_2
  6. DataFrame generated for file input_pdf\2013\ns-06-2013.pdf: ns_06_2013_2
  7. DataFrame generated for file input_pdf\2013\ns-07-2013.pdf: ns_07_2013_2
  8. DataFrame generated for file input_pdf\2013\ns-08-2013.pdf: ns_08_2013_2
  9. DataFrame generated for file input_pdf\2013\ns-09-2013.pdf: ns_09_2013_2
  10. DataFrame generated for file input_pdf\2013\ns-10-2013.pdf: ns_10_2013_2
  11. DataFrame generated for file input_pdf\2013\ns-11-2013.pdf: ns_11_2013_2
  12. DataFrame generated for file input_pdf\2013\ns-12-2013.pdf: ns_12_2013_2
  13. DataFrame generated for file inp

  6. DataFrame generated for file input_pdf\2015\ns-06-2015.pdf: ns_06_2015_2
  7. DataFrame generated for file input_pdf\2015\ns-07-2015.pdf: ns_07_2015_2
  8. DataFrame generated for file input_pdf\2015\ns-08-2015.pdf: ns_08_2015_2
  9. DataFrame generated for file input_pdf\2015\ns-09-2015.pdf: ns_09_2015_2
  10. DataFrame generated for file input_pdf\2015\ns-10-2015.pdf: ns_10_2015_2
  11. DataFrame generated for file input_pdf\2015\ns-11-2015.pdf: ns_11_2015_2
  12. DataFrame generated for file input_pdf\2015\ns-12-2015.pdf: ns_12_2015_2
  13. DataFrame generated for file input_pdf\2015\ns-13-2015.pdf: ns_13_2015_2
  14. DataFrame generated for file input_pdf\2015\ns-14-2015.pdf: ns_14_2015_2
  15. DataFrame generated for file input_pdf\2015\ns-15-2015.pdf: ns_15_2015_2
  16. DataFrame generated for file input_pdf\2015\ns-16-2015.pdf: ns_16_2015_2
  17. DataFrame generated for file input_pdf\2015\ns-17-2015.pdf: ns_17_2015_2
  18. DataFrame generated for file input_pdf\2015\ns-18-

  14. DataFrame generated for file input_pdf\2017\ns-14-2017.pdf: ns_14_2017_2
  15. DataFrame generated for file input_pdf\2017\ns-15-2017.pdf: ns_15_2017_2
  16. DataFrame generated for file input_pdf\2017\ns-16-2017.pdf: ns_16_2017_2
  17. DataFrame generated for file input_pdf\2017\ns-17-2017.pdf: ns_17_2017_2
  18. DataFrame generated for file input_pdf\2017\ns-18-2017.pdf: ns_18_2017_2
  19. DataFrame generated for file input_pdf\2017\ns-19-2017.pdf: ns_19_2017_2
  20. DataFrame generated for file input_pdf\2017\ns-20-2017.pdf: ns_20_2017_2
  21. DataFrame generated for file input_pdf\2017\ns-21-2017.pdf: ns_21_2017_2
  22. DataFrame generated for file input_pdf\2017\ns-22-2017.pdf: ns_22_2017_2
  23. DataFrame generated for file input_pdf\2017\ns-23-2017.pdf: ns_23_2017_2
  24. DataFrame generated for file input_pdf\2017\ns-24-2017.pdf: ns_24_2017_2
  25. DataFrame generated for file input_pdf\2017\ns-25-2017.pdf: ns_25_2017_2
  26. DataFrame generated for file input_pdf\2017\ns

  20. DataFrame generated for file input_pdf\2019\ns-20-2019.pdf: ns_20_2019_2
  21. DataFrame generated for file input_pdf\2019\ns-21-2019.pdf: ns_21_2019_2
  22. DataFrame generated for file input_pdf\2019\ns-22-2019.pdf: ns_22_2019_2
  23. DataFrame generated for file input_pdf\2019\ns-23-2019.pdf: ns_23_2019_2
  24. DataFrame generated for file input_pdf\2019\ns-24-2019.pdf: ns_24_2019_2
  25. DataFrame generated for file input_pdf\2019\ns-25-2019.pdf: ns_25_2019_2
  26. DataFrame generated for file input_pdf\2019\ns-26-2019.pdf: ns_26_2019_2
  27. DataFrame generated for file input_pdf\2019\ns-27-2019.pdf: ns_27_2019_2
  28. DataFrame generated for file input_pdf\2019\ns-28-2019.pdf: ns_28_2019_2
  29. DataFrame generated for file input_pdf\2019\ns-29-2019.pdf: ns_29_2019_2
  30. DataFrame generated for file input_pdf\2019\ns-30-2019.pdf: ns_30_2019_2
  31. DataFrame generated for file input_pdf\2019\ns-31-2019.pdf: ns_31_2019_2
  32. DataFrame generated for file input_pdf\2019\ns

  38. DataFrame generated for file input_pdf\2021\ns-38-2021.pdf: ns_38_2021_2
  39. DataFrame generated for file input_pdf\2021\ns-39-2021.pdf: ns_39_2021_2
  40. DataFrame generated for file input_pdf\2021\ns-40-2021.pdf: ns_40_2021_2
  41. DataFrame generated for file input_pdf\2021\ns-41-2021.pdf: ns_41_2021_2
  42. DataFrame generated for file input_pdf\2021\ns-42-2021.pdf: ns_42_2021_2
  43. DataFrame generated for file input_pdf\2021\ns-43-2021.pdf: ns_43_2021_2
  44. DataFrame generated for file input_pdf\2021\ns-44-2021.pdf: ns_44_2021_2
  45. DataFrame generated for file input_pdf\2021\ns-45-2021.pdf: ns_45_2021_2
  46. DataFrame generated for file input_pdf\2021\ns-46-2021.pdf: ns_46_2021_2
Processing folder 2022
  1. DataFrame generated for file input_pdf\2022\ns-01-2022.pdf: ns_01_2022_2
  2. DataFrame generated for file input_pdf\2022\ns-02-2022.pdf: ns_02_2022_2
  3. DataFrame generated for file input_pdf\2022\ns-03-2022.pdf: ns_03_2022_2
  4. DataFrame generated for fil

  9. DataFrame generated for file input_pdf\2024\ns-09-2024.pdf: ns_09_2024_2
  10. DataFrame generated for file input_pdf\2024\ns-10-2024.pdf: ns_10_2024_2
  11. DataFrame generated for file input_pdf\2024\ns-11-2024.pdf: ns_11_2024_2
  12. DataFrame generated for file input_pdf\2024\ns-12-2024.pdf: ns_12_2024_2
  13. DataFrame generated for file input_pdf\2024\ns-13-2024.pdf: ns_13_2024_2
  14. DataFrame generated for file input_pdf\2024\ns-14-2024.pdf: ns_14_2024_2
  15. DataFrame generated for file input_pdf\2024\ns-15-2024.pdf: ns_15_2024_2
  16. DataFrame generated for file input_pdf\2024\ns-16-2024.pdf: ns_16_2024_2
  17. DataFrame generated for file input_pdf\2024\ns-17-2024.pdf: ns_17_2024_2
  18. DataFrame generated for file input_pdf\2024\ns-18-2024.pdf: ns_18_2024_2
  19. DataFrame generated for file input_pdf\2024\ns-19-2024.pdf: ns_19_2024_2
  20. DataFrame generated for file input_pdf\2024\ns-20-2024.pdf: ns_20_2024_2
  21. DataFrame generated for file input_pdf\2024\ns-

In [20]:
new_tables_dict_2.keys()

dict_keys(['ns_01_2013_2', 'ns_02_2013_2', 'ns_03_2013_2', 'ns_04_2013_2', 'ns_05_2013_2', 'ns_06_2013_2', 'ns_07_2013_2', 'ns_08_2013_2', 'ns_09_2013_2', 'ns_10_2013_2', 'ns_11_2013_2', 'ns_12_2013_2', 'ns_13_2013_2', 'ns_14_2013_2', 'ns_15_2013_2', 'ns_16_2013_2', 'ns_17_2013_2', 'ns_18_2013_2', 'ns_19_2013_2', 'ns_20_2013_2', 'ns_21_2013_2', 'ns_22_2013_2', 'ns_23_2013_2', 'ns_24_2013_2', 'ns_25_2013_2', 'ns_26_2013_2', 'ns_27_2013_2', 'ns_28_2013_2', 'ns_29_2013_2', 'ns_30_2013_2', 'ns_31_2013_2', 'ns_32_2013_2', 'ns_33_2013_2', 'ns_34_2013_2', 'ns_35_2013_2', 'ns_36_2013_2', 'ns_37_2013_2', 'ns_38_2013_2', 'ns_39_2013_2', 'ns_40_2013_2', 'ns_41_2013_2', 'ns_42_2013_2', 'ns_43_2013_2', 'ns_44_2013_2', 'ns_45_2013_2', 'ns_46_2013_2', 'ns_47_2013_2', 'ns_48_2013_2', 'ns_49_2013_2', 'ns_50_2013_2', 'ns_01_2014_2', 'ns_02_2014_2', 'ns_03_2014_2', 'ns_04_2014_2', 'ns_05_2014_2', 'ns_06_2014_2', 'ns_07_2014_2', 'ns_08_2014_2', 'ns_09_2014_2', 'ns_10_2014_2', 'ns_11_2014_2', 'ns_12_2014_2

In [21]:
new_dataframes_dict_2.keys()

dict_keys(['ns_01_2013_2', 'ns_02_2013_2', 'ns_03_2013_2', 'ns_04_2013_2', 'ns_05_2013_2', 'ns_06_2013_2', 'ns_07_2013_2', 'ns_08_2013_2', 'ns_09_2013_2', 'ns_10_2013_2', 'ns_11_2013_2', 'ns_12_2013_2', 'ns_13_2013_2', 'ns_14_2013_2', 'ns_15_2013_2', 'ns_16_2013_2', 'ns_17_2013_2', 'ns_18_2013_2', 'ns_19_2013_2', 'ns_20_2013_2', 'ns_21_2013_2', 'ns_22_2013_2', 'ns_23_2013_2', 'ns_24_2013_2', 'ns_25_2013_2', 'ns_26_2013_2', 'ns_27_2013_2', 'ns_28_2013_2', 'ns_29_2013_2', 'ns_30_2013_2', 'ns_31_2013_2', 'ns_32_2013_2', 'ns_33_2013_2', 'ns_34_2013_2', 'ns_35_2013_2', 'ns_36_2013_2', 'ns_37_2013_2', 'ns_38_2013_2', 'ns_39_2013_2', 'ns_40_2013_2', 'ns_41_2013_2', 'ns_42_2013_2', 'ns_43_2013_2', 'ns_44_2013_2', 'ns_45_2013_2', 'ns_46_2013_2', 'ns_47_2013_2', 'ns_48_2013_2', 'ns_49_2013_2', 'ns_50_2013_2', 'ns_01_2014_2', 'ns_02_2014_2', 'ns_03_2014_2', 'ns_04_2014_2', 'ns_05_2014_2', 'ns_06_2014_2', 'ns_07_2014_2', 'ns_08_2014_2', 'ns_09_2014_2', 'ns_10_2014_2', 'ns_11_2014_2', 'ns_12_2014_2

In [22]:
new_tables_dict_2['ns_43_2024_2'].head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,2022,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,2023,Unnamed: 10,Unnamed: 11,Unnamed: 12,2024,Unnamed: 14,Unnamed: 15
0,SECTORES ECONÓMICOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ECONOMIC SECTORS
1,NaN,I,NaN,II,III,IV,AÑO,I,II,III,IV,AÑO,I,II,III,NaN
2,Agropecuario,"4,7",NaN,"4,9","5,4","3,1","4,6","1,4","-5,6","-3,3","-0,5","-2,4","-0,5","8,5","-1,5 Agriculture and Livestock",NaN
3,Pesca,"-22,7",NaN,"-7,8","17,4","-17,7","-11,4","16,3","-61,5","-11,9","-6,4","-21,2","-29,8","184,2","-11,1 Fishing",NaN
4,Minería e hidrocarburos,"1,3",NaN,"-1,1","-4,2","6,0","0,5","2,6","16,5","8,8","4,8","8,1","8,2","-3,1","4,4 Mining and fuel",NaN


In [23]:
new_dataframes_dict_2['ns_43_2024_2']

,year,id_ns,date,sectores_economicos,economic_sectors,2022_1,2022_2,2022_3,2022_4,2022_year,2023_1,2023_2,2023_3,2023_4,2023_year,2024_1,2024_2,2024_3
0,2024,43,2024-11-28,agropecuario,agriculture and livestock,4.7,4.9,5.4,3.1,4.6,1.4,-5.6,-3.3,-0.5,-2.4,-0.5,8.5,-1.5
1,2024,43,2024-11-28,pesca,fishing,-22.7,-7.8,17.4,-17.7,-11.4,16.3,-61.5,-11.9,-6.4,-21.2,-29.8,184.2,-11.1
2,2024,43,2024-11-28,mineria e hidrocarburos,mining and fuel,1.3,-1.1,-4.2,6.0,0.5,2.6,16.5,8.8,4.8,8.1,8.2,-3.1,4.4
3,2024,43,2024-11-28,manufactura,manufacturing,1.6,3.4,1.2,-2.2,1.0,-0.4,-11.7,-8.3,-5.4,-6.5,-6.4,10.0,5.3
4,2024,43,2024-11-28,electricidad y agua,electricity and water,3.4,2.6,4.3,5.2,3.9,4.6,5.8,3.0,1.5,3.7,3.7,1.4,2.2
5,2024,43,2024-11-28,construccion,construction,-0.4,3.4,3.6,5.1,3.1,-11.4,-7.2,-9.1,-4.5,-7.8,4.6,3.1,4.7
6,2024,43,2024-11-28,comercio,commerce,5.9,2.6,2.4,2.5,3.3,2.2,3.1,2.6,1.6,2.4,2.4,2.5,3.2
7,2024,43,2024-11-28,otros servicios,other services,5.4,4.6,2.9,1.6,3.6,-0.3,0.2,-0.3,0.0,-0.1,1.6,2.7,4.1
8,2024,43,2024-11-28,pbi global,gdp,3.9,3.4,2.0,2.0,2.8,-0.1,-0.5,-0.7,-0.3,-0.4,1.4,3.6,3.8
9,2024,43,2024-11-28,sectores primarios,primary sectors,-0.1,0.0,-0.3,4.0,0.9,5.1,1.3,3.9,1.6,2.9,0.9,7.5,2.4


In [ ]:
# Call the function to set the counter
if frequency == "monthly":
    counter = 1
elif frequency == "quarterly":
    counter = 2
elif frequency == "annual":
    counter = 2
else:
    counter = None 

print(counter)

In [ ]:
# Dynamically construct the function name and dictionary name
function_name = f"concatenate_{frequency}_df"
dataframe_dict_name = f"new_dataframes_dict_{counter}"

# Check that both the function and dictionary exist in the global scope
if function_name in globals() and dataframe_dict_name in globals():
    # Call the function using its reference from globals()
    globals()[f"new_{sector}_{frequency}_growth_rates"] = globals()[function_name](
        globals()[dataframe_dict_name], selected_spanish, selected_english
    )
else:
    print(f"Error: {function_name} or {dataframe_dict_name} does not exist in the global scope.")

In [ ]:
engine = create_sqlalchemy_engine()

In [ ]:
globals()[f"new_{sector}_{frequency}_growth_rates"].to_sql(f'new_{sector}_{frequency}_growth_rates', engine, index=False, if_exists='replace')

In [24]:
frequencies = [
        "monthly", 
        "quarterly",
        "annual"
    ]

In [25]:
def process_new_datasets_to_sql(sector, frequency):
    # Initialize counter for loaded DataFrames
    total_loaded = 0

    # Set counter based on frequency
    if frequency == "monthly":
        counter = 1
    elif frequency in ["quarterly", "annual"]:
        counter = 2
    else:
        print(f"Unknown frequency: {frequency}")
        return None

    # Dynamically build function and dictionary names
    function_name = f"concatenate_{frequency}_df"
    dataframe_dict_name = f"new_dataframes_dict_{counter}"

    if function_name in globals() and dataframe_dict_name in globals():
        # Generate the DataFrame
        df_name = f"new_{sector}_{frequency}_growth_rates"
        globals()[df_name] = globals()[function_name](
            globals()[dataframe_dict_name], option_mapping[sector][0], option_mapping[sector][1]
        )

        # Load to SQL
        engine = create_sqlalchemy_engine()
        globals()[df_name].to_sql(df_name, engine, index=False, if_exists='replace')

        return globals()[df_name]
    else:
        print(f"Error: {function_name} or {dataframe_dict_name} does not exist in the global scope.")
        return None

In [26]:
# Initialize counter
processed_datasets = 0

# Process all combinations
for sector in option_mapping.keys():
    for frequency in frequencies:
        print(f"Processing {sector} - {frequency}")
        df = process_new_datasets_to_sql(sector, frequency)
        if df is not None:
            display(df.head(10))  # Display the first 10 rows
            processed_datasets += 1  # Increment counter

# Display total number of processed datasets
print(f"Total datasets processed: {processed_datasets}")

Processing gdp - monthly
Number of rows in the concatenated dataframe: 555


,year,id_ns,date,sep_2011,oct_2011,nov_2011,dic_2011,ene_2012,feb_2012,mar_2012,...,dic_2023,ene_2024,feb_2024,mar_2024,abr_2024,may_2024,jun_2024,jul_2024,ago_2024,sep_2024
0,2013,01,2013-01-04,5.9,5.3,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,02,2013-01-11,5.9,5.3,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,03,2013-01-18,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,04,2013-01-25,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,05,2013-02-01,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013,06,2013-02-08,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,07,2013-02-15,NaN,NaN,5.1,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013,08,2013-02-22,NaN,NaN,NaN,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013,09,2013-03-01,NaN,NaN,NaN,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2013,10,2013-03-08,NaN,NaN,NaN,6.0,5.5,6.9,5.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Processing gdp - quarterly
Number of rows in the concatenated dataframe: 555


,year,id_ns,date,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,...,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1,2024_2,2024_3
0,2013,01,2013-01-04,6.2,10.0,9.6,9.2,8.8,6.9,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,02,2013-01-11,6.2,10.0,9.6,9.2,8.8,6.9,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,03,2013-01-18,6.2,10.0,9.6,9.2,8.8,6.9,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,04,2013-01-25,6.2,10.0,9.6,9.2,8.8,6.9,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,05,2013-02-01,6.2,10.0,9.6,9.2,8.8,6.9,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013,06,2013-02-08,6.2,10.0,9.6,9.2,8.8,6.9,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,07,2013-02-15,6.2,10.0,9.6,9.2,8.8,6.9,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013,08,2013-02-22,6.2,10.0,9.6,9.2,8.8,6.9,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013,09,2013-03-01,6.2,10.0,9.6,9.2,8.8,6.9,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2013,10,2013-03-08,6.2,10.0,9.6,9.2,8.8,6.9,6.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Processing gdp - annual
Number of rows in the concatenated dataframe: 555


...

Processing services - quarterly
Number of rows in the concatenated dataframe: 555


,year,id_ns,date,2010_1,2010_2,2010_3,2010_4,2011_1,2011_2,2011_3,...,2022_2,2022_3,2022_4,2023_1,2023_2,2023_3,2023_4,2024_1,2024_2,2024_3
0,2013,01,2013-01-04,4.9,8.8,9.3,8.9,9.3,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,02,2013-01-11,4.9,8.8,9.3,8.9,9.3,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,03,2013-01-18,4.9,8.8,9.3,8.9,9.3,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,04,2013-01-25,4.9,8.8,9.3,8.9,9.3,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,05,2013-02-01,4.9,8.8,9.3,8.9,9.3,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013,06,2013-02-08,4.9,8.8,9.3,8.9,9.3,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,07,2013-02-15,4.9,8.8,9.3,8.9,9.3,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013,08,2013-02-22,4.9,8.8,9.3,8.9,9.3,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013,09,2013-03-01,4.9,8.8,9.3,8.9,9.3,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2013,10,2013-03-08,4.9,8.8,9.3,8.9,9.3,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Processing services - annual
Number of rows in the concatenated dataframe: 555


,year,id_ns,date,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023
0,2013,01,2013-01-04,8.0,8.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,02,2013-01-11,8.0,8.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,03,2013-01-18,8.0,8.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013,04,2013-01-25,8.0,8.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013,05,2013-02-01,8.0,8.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013,06,2013-02-08,8.0,8.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2013,07,2013-02-15,8.0,8.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013,08,2013-02-22,8.0,8.3,7.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013,09,2013-03-01,8.0,8.3,7.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2013,10,2013-03-08,8.0,8.3,7.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Total datasets processed: 27


---
---
